In [1]:
import os
import sys
from time import gmtime, strftime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import torch
import poutyne
from poutyne import Model,Experiment

In [3]:
from data.custom_data import filepath_dataframe
from data.selection import Selection,SelectionSet_1
from data.transformation import Transform_CnnLstmS,Transform_CnnS,Transform_Cnn
from data.torchData import DataLoadings,DataLoading

from data.custom_data import nucPaired_fpDataframe
from data.torchData import PairDataLoading,DataLoading
from training.contrastive_pretraining import Contrastive_PreTraining
from training.finetuning import FineTuneCNN

import models

In [4]:
#####################################################################################################################

# random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# gpu setting
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(DEVICE)
device = DEVICE

## data directory
data_dir  = 'E:\\external_data\\opera_csi\\Session_2\\experiment_data\\experiment_data\\exp_7_amp_spec_only\\npy_format'
readtype = 'npy'
splitchar = '\\'
fpath = '.'

# data selection
data_selection = SelectionSet_1() # Selection(split='loov',test_sub='Three',val_sub=0.1, nuc='NUC1',room=1,sample_per_class=None)
dataselection_name = 'SelectionSet1'

# data loading
transform = None
batch_size = 64
num_workers = 0

# training
optimizer_builder = torch.optim.Adam
lr = 0.0005
pretrain_epochs = 1
finetune_epochs = 300

# model
builder = lambda : (models.baseline.ConvNet1D(strides=[16,8,4]), 6144) # strides=[16,8,4]
hidden_layer = 128
network_name = 'ConvNet1D'

# Experiment Name
comment = '3LayerDefault'


# auto
exp_name = f'{network_name}_Supervised_{dataselection_name}_Comment-{comment}'
record_dir = os.path.join(fpath,'records')
model_dir = os.path.join(fpath,'saved_model')
model_fname = os.path.join(model_dir,f'Encoder___{exp_name}')
record_fname =  os.path.join(record_dir,f'{exp_name}.csv')
print('Experiment Name: ',exp_name)
print('Cuda Availability: ',torch.cuda.is_available())

Experiment Name:  ConvNet1D_Supervised_SelectionSet1_Comment-3LayerDefault
Cuda Availability:  True


In [5]:
# -----------------------------------Main-------------------------------------------

# if __name__ == '__main__':

# data preparation
df = filepath_dataframe(data_dir,splitchar)
df_train,df_val,df_test = data_selection(df)
df_train = pd.concat([df_train,df_val])

In [6]:
##### FINE-TUNING #####

# data loading
data_loading = DataLoading(transform=transform,batch_size=batch_size,readtype=readtype,
                           num_workers=num_workers,drop_last=True)
test_loading = DataLoading(transform=transform,batch_size=len(df_test),readtype=readtype,
                           num_workers=num_workers,drop_last=True)

train_loader = data_loading(df_train)
test_loader  = test_loading(df_test)

# load and create model
model = FineTuneCNN(model_path=None,
                    encoder_builder=builder,
                    n_classes=df.activity.nunique(),
                    hidden_layer=hidden_layer)

In [7]:
from torchsummary import summary

summary(model,(70,6400),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 128, 397]         573,568
       BatchNorm1d-2             [-1, 128, 397]               0
            Conv1d-3              [-1, 256, 48]         524,544
       BatchNorm1d-4              [-1, 256, 48]               0
            Conv1d-5              [-1, 512, 12]         524,800
       BatchNorm1d-6              [-1, 512, 12]               0
           Flatten-7                 [-1, 6144]               0
         ConvNet1D-8                 [-1, 6144]               0
            Linear-9                  [-1, 128]         786,560
        LeakyReLU-10                  [-1, 128]               0
          Dropout-11                  [-1, 128]               0
           Linear-12                    [-1, 6]             774
       Classifier-13                    [-1, 6]               0
Total params: 2,410,246
Trainable param

In [ ]:
# train with poutyne
mdl = Model(model,'adam','cross_entropy',
            batch_metrics=['accuracy'],
            epoch_metrics=[poutyne.F1('micro'),poutyne.F1('macro')]).to(device)
history = mdl.fit_generator(train_generator=train_loader,valid_generator=test_loader,epochs=finetune_epochs)

Epoch: 1/300 Step: 22/22 100.00% |█████████████████████████|13.41s loss: 5.362283 acc: 27.698864 fscore_micro: 0.276989 fscore_macro: 0.125666 val_loss: 2.019791 val_acc: 25.757576 val_fscore_micro: 0.257576 val_fscore_macro: 0.182119
Epoch: 2/300 Step: 22/22 100.00% |█████████████████████████|12.88s loss: 1.825870 acc: 27.911932 fscore_micro: 0.279119 fscore_macro: 0.083249 val_loss: 1.874961 val_acc: 17.424242 val_fscore_micro: 0.174242 val_fscore_macro: 0.070608
Epoch: 3/300 Step: 22/22 100.00% |█████████████████████████|14.05s loss: 1.697204 acc: 31.321023 fscore_micro: 0.313210 fscore_macro: 0.122568 val_loss: 1.866872 val_acc: 17.424242 val_fscore_micro: 0.174242 val_fscore_macro: 0.071989
Epoch: 4/300 Step: 22/22 100.00% |█████████████████████████|19.17s loss: 1.645831 acc: 32.386364 fscore_micro: 0.323864 fscore_macro: 0.146432 val_loss: 1.835785 val_acc: 18.181818 val_fscore_micro: 0.181818 val_fscore_macro: 0.107442
Epoch: 5/300 Step: 22/22 100.00% |█████████████████████████|

Epoch: 24/300 Step: 22/22 100.00% |█████████████████████████|13.86s loss: 1.028631 acc: 57.457386 fscore_micro: 0.574574 fscore_macro: 0.580397 val_loss: 1.635183 val_acc: 46.212120 val_fscore_micro: 0.462121 val_fscore_macro: 0.454124
Epoch: 25/300 Step: 22/22 100.00% |█████████████████████████|18.16s loss: 1.054567 acc: 55.894886 fscore_micro: 0.558949 fscore_macro: 0.554625 val_loss: 1.901995 val_acc: 44.696968 val_fscore_micro: 0.446970 val_fscore_macro: 0.465119
Epoch: 26/300 Step: 22/22 100.00% |█████████████████████████|12.85s loss: 1.110745 acc: 54.190341 fscore_micro: 0.541903 fscore_macro: 0.521133 val_loss: 1.580997 val_acc: 44.696968 val_fscore_micro: 0.446970 val_fscore_macro: 0.437825
Epoch: 27/300 Step: 22/22 100.00% |█████████████████████████|12.84s loss: 1.047770 acc: 57.954545 fscore_micro: 0.579545 fscore_macro: 0.569916 val_loss: 1.560303 val_acc: 50.000000 val_fscore_micro: 0.500000 val_fscore_macro: 0.500155
Epoch: 28/300 Step: 22/22 100.00% |█████████████████████

Epoch: 47/300 Step: 22/22 100.00% |█████████████████████████|12.70s loss: 0.423865 acc: 84.517045 fscore_micro: 0.845170 fscore_macro: 0.845812 val_loss: 2.799918 val_acc: 34.090908 val_fscore_micro: 0.340909 val_fscore_macro: 0.331275
Epoch: 48/300 Step: 22/22 100.00% |█████████████████████████|13.65s loss: 0.495933 acc: 78.409091 fscore_micro: 0.784091 fscore_macro: 0.778150 val_loss: 3.288219 val_acc: 30.303032 val_fscore_micro: 0.303030 val_fscore_macro: 0.288576
Epoch: 49/300 Step: 22/22 100.00% |█████████████████████████|15.29s loss: 0.431835 acc: 83.025568 fscore_micro: 0.830256 fscore_macro: 0.839309 val_loss: 2.921170 val_acc: 37.878788 val_fscore_micro: 0.378788 val_fscore_macro: 0.357054
Epoch: 50/300 Step: 22/22 100.00% |█████████████████████████|16.16s loss: 0.444169 acc: 82.528409 fscore_micro: 0.825284 fscore_macro: 0.824402 val_loss: 2.411667 val_acc: 34.848484 val_fscore_micro: 0.348485 val_fscore_macro: 0.314900
Epoch: 51/300 Step: 22/22 100.00% |█████████████████████

Epoch: 70/300 Step: 22/22 100.00% |█████████████████████████|11.89s loss: 0.187044 acc: 93.536932 fscore_micro: 0.935369 fscore_macro: 0.937039 val_loss: 3.402895 val_acc: 34.848484 val_fscore_micro: 0.348485 val_fscore_macro: 0.334606
Epoch: 71/300 Step: 22/22 100.00% |█████████████████████████|11.99s loss: 0.190732 acc: 93.963068 fscore_micro: 0.939631 fscore_macro: 0.934849 val_loss: 4.937306 val_acc: 31.060606 val_fscore_micro: 0.310606 val_fscore_macro: 0.299074
Epoch: 72/300 Step: 22/22 100.00% |█████████████████████████|11.91s loss: 0.160890 acc: 95.028409 fscore_micro: 0.950284 fscore_macro: 0.954318 val_loss: 4.958186 val_acc: 29.545454 val_fscore_micro: 0.295455 val_fscore_macro: 0.287594
Epoch: 73/300 Step: 22/22 100.00% |█████████████████████████|11.95s loss: 0.141015 acc: 95.809659 fscore_micro: 0.958097 fscore_macro: 0.957678 val_loss: 5.166992 val_acc: 29.545454 val_fscore_micro: 0.295455 val_fscore_macro: 0.289116
Epoch: 74/300 Step: 22/22 100.00% |█████████████████████

Epoch: 93/300 Step: 22/22 100.00% |█████████████████████████|11.94s loss: 0.061373 acc: 98.153409 fscore_micro: 0.981534 fscore_macro: 0.979182 val_loss: 5.285191 val_acc: 31.060606 val_fscore_micro: 0.310606 val_fscore_macro: 0.290889
Epoch: 94/300 Step: 22/22 100.00% |█████████████████████████|11.92s loss: 0.058112 acc: 98.579545 fscore_micro: 0.985795 fscore_macro: 0.984720 val_loss: 5.774714 val_acc: 29.545454 val_fscore_micro: 0.295455 val_fscore_macro: 0.276617
Epoch: 95/300 Step: 22/22 100.00% |█████████████████████████|12.06s loss: 0.064050 acc: 98.011364 fscore_micro: 0.980114 fscore_macro: 0.976355 val_loss: 6.016749 val_acc: 32.575756 val_fscore_micro: 0.325758 val_fscore_macro: 0.309020
Epoch: 96/300 Step: 22/22 100.00% |█████████████████████████|11.92s loss: 0.045655 acc: 98.792614 fscore_micro: 0.987926 fscore_macro: 0.988929 val_loss: 5.400718 val_acc: 29.545454 val_fscore_micro: 0.295455 val_fscore_macro: 0.288734
Epoch: 97/300 Step: 22/22 100.00% |█████████████████████

...

Epoch: 162/300 Step: 22/22 100.00% |█████████████████████████|14.40s loss: 0.012858 acc: 99.786932 fscore_micro: 0.997869 fscore_macro: 0.997355 val_loss: 5.829240 val_acc: 33.333336 val_fscore_micro: 0.333333 val_fscore_macro: 0.301497
Epoch: 163/300 Step: 22/22 100.00% |█████████████████████████|13.49s loss: 0.013446 acc: 99.857955 fscore_micro: 0.998580 fscore_macro: 0.999222 val_loss: 6.021155 val_acc: 31.818182 val_fscore_micro: 0.318182 val_fscore_macro: 0.289277
Epoch: 164/300 Step: 22/22 100.00% |█████████████████████████|12.10s loss: 0.016058 acc: 99.715909 fscore_micro: 0.997159 fscore_macro: 0.997553 val_loss: 6.082862 val_acc: 34.090908 val_fscore_micro: 0.340909 val_fscore_macro: 0.302787
Epoch: 165/300 Step: 22/22 100.00% |█████████████████████████|12.07s loss: 0.016849 acc: 99.644886 fscore_micro: 0.996449 fscore_macro: 0.996272 val_loss: 6.198305 val_acc: 32.575756 val_fscore_micro: 0.325758 val_fscore_macro: 0.291516
Epoch: 166/300 Step: 22/22 100.00% |████████████████

Epoch: 185/300 Step: 22/22 100.00% |█████████████████████████|11.95s loss: 0.021325 acc: 99.502841 fscore_micro: 0.995028 fscore_macro: 0.995484 val_loss: 5.917440 val_acc: 28.030304 val_fscore_micro: 0.280303 val_fscore_macro: 0.260097
Epoch: 186/300 Step: 22/22 100.00% |█████████████████████████|12.21s loss: 0.030133 acc: 99.218750 fscore_micro: 0.992188 fscore_macro: 0.991183 val_loss: 6.224096 val_acc: 37.121212 val_fscore_micro: 0.371212 val_fscore_macro: 0.313702
Epoch: 187/300 Step: 22/22 100.00% |█████████████████████████|12.79s loss: 0.057211 acc: 98.224432 fscore_micro: 0.982244 fscore_macro: 0.985548 val_loss: 8.567672 val_acc: 25.757576 val_fscore_micro: 0.257576 val_fscore_macro: 0.253844
Epoch: 188/300 Step: 22/22 100.00% |█████████████████████████|12.05s loss: 0.101988 acc: 96.164773 fscore_micro: 0.961648 fscore_macro: 0.965819 val_loss: 8.712842 val_acc: 31.060606 val_fscore_micro: 0.310606 val_fscore_macro: 0.288259
Epoch: 189/300 Step: 22/22 100.00% |████████████████

Epoch: 208/300 Step: 22/22 100.00% |█████████████████████████|14.44s loss: 0.050025 acc: 98.508523 fscore_micro: 0.985085 fscore_macro: 0.981339 val_loss: 6.164179 val_acc: 28.030304 val_fscore_micro: 0.280303 val_fscore_macro: 0.261578
Epoch: 209/300 Step: 22/22 100.00% |█████████████████████████|14.54s loss: 0.042719 acc: 98.721591 fscore_micro: 0.987216 fscore_macro: 0.987024 val_loss: 6.495796 val_acc: 30.303032 val_fscore_micro: 0.303030 val_fscore_macro: 0.287198
Epoch: 210/300 Step: 22/22 100.00% |█████████████████████████|12.07s loss: 0.035947 acc: 98.863636 fscore_micro: 0.988636 fscore_macro: 0.987460 val_loss: 6.258346 val_acc: 31.060606 val_fscore_micro: 0.310606 val_fscore_macro: 0.292068
Epoch: 211/300 Step: 22/22 100.00% |█████████████████████████|12.46s loss: 0.030937 acc: 99.360795 fscore_micro: 0.993608 fscore_macro: 0.991733 val_loss: 6.378889 val_acc: 30.303032 val_fscore_micro: 0.303030 val_fscore_macro: 0.277161
Epoch: 212/300 Step: 22/22 100.00% |████████████████

Epoch: 231/300 Step: 22/22 100.00% |█████████████████████████|12.68s loss: 0.020127 acc: 99.431818 fscore_micro: 0.994318 fscore_macro: 0.994207 val_loss: 7.437157 val_acc: 30.303032 val_fscore_micro: 0.303030 val_fscore_macro: 0.288616
Epoch: 232/300 Step: 22/22 100.00% |█████████████████████████|12.01s loss: 0.014552 acc: 99.573864 fscore_micro: 0.995739 fscore_macro: 0.994657 val_loss: 7.621285 val_acc: 28.030304 val_fscore_micro: 0.280303 val_fscore_macro: 0.260645
Epoch: 233/300 Step: 22/22 100.00% |█████████████████████████|12.01s loss: 0.024090 acc: 99.715909 fscore_micro: 0.997159 fscore_macro: 0.997837 val_loss: 7.268957 val_acc: 31.818182 val_fscore_micro: 0.318182 val_fscore_macro: 0.297564
Epoch: 234/300 Step: 22/22 100.00% |█████████████████████████|12.81s loss: 0.007672 acc: 99.786932 fscore_micro: 0.997869 fscore_macro: 0.996757 val_loss: 7.512491 val_acc: 35.606060 val_fscore_micro: 0.356061 val_fscore_macro: 0.330010
Epoch: 235/300 Step: 22/22 100.00% |████████████████

Epoch: 254/300 Step: 22/22 100.00% |█████████████████████████|12.87s loss: 0.012756 acc: 99.573864 fscore_micro: 0.995739 fscore_macro: 0.994411 val_loss: 8.417929 val_acc: 31.818182 val_fscore_micro: 0.318182 val_fscore_macro: 0.289500
Epoch: 255/300 Step: 22/22 100.00% |█████████████████████████|13.82s loss: 0.039493 acc: 98.863636 fscore_micro: 0.988636 fscore_macro: 0.984803 val_loss: 6.660084 val_acc: 25.000000 val_fscore_micro: 0.250000 val_fscore_macro: 0.251397
Epoch: 256/300 Step: 22/22 100.00% |█████████████████████████|12.55s loss: 0.167953 acc: 95.099432 fscore_micro: 0.950994 fscore_macro: 0.934337 val_loss: 8.850465 val_acc: 28.787878 val_fscore_micro: 0.287879 val_fscore_macro: 0.265239
Epoch: 257/300 Step: 22/22 100.00% |█████████████████████████|12.00s loss: 0.284091 acc: 90.767045 fscore_micro: 0.907670 fscore_macro: 0.887001 val_loss: 8.647546 val_acc: 34.090908 val_fscore_micro: 0.340909 val_fscore_macro: 0.334937
Epoch: 258/300 Step: 22/22 100.00% |████████████████

In [ ]:
pd.DataFrame(history).to_csv(record_fname)